# Settings

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab_Projects/21_F_자연어처리_이병준
!ls

/content/drive/MyDrive/Colab_Projects/21_F_자연어처리_이병준
'Datasets_&_Weight'   Docs    ResNet_Implementation.ipynb
 Deep_ResNet.ipynb    Model


In [3]:
from fastai import *
from fastai.text import *
from nltk.tokenize import word_tokenize
from fastai.callbacks import *
import torch.utils.data as data_utils

!pip install import_ipynb

import import_ipynb
from Deep_ResNet import *

importing Jupyter notebook from Deep_ResNet.ipynb


# Training Session

In [4]:
defaults.device = torch.device('cuda')

In [5]:
PADDING = 64  
DATASET_PATH = './Datasets_&_Weight/'
DATASET = 'Headlines.csv'  
COL = 'headline'  

WEIGHTS = './Datasets_&_Weight/Weights_glove_headlines.pkl'
EMBED_DIM = 50 
STATIC = False 

In [6]:
def pad_to(x:Collection[str], pad_til = PADDING) -> Collection[str]:
  res = []
  count = 0
  
  for t in x:
    res.append(t)
    count += 1
  while count < pad_til:
    res.append(PAD)
    count +=1
    
  return res

In [7]:
tokenizer = Tokenizer(SpacyTokenizer, 'en', 
                      pre_rules=[fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces], 
                      post_rules=[replace_all_caps, deal_caps, pad_to])

processor = [TokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor()]

In [8]:
data = (TextList.from_csv(DATASET_PATH, DATASET, cols=COL, processor=processor))

In [9]:
data = data.split_from_df(col='valid').label_from_df(cols=0).databunch()

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
weights_matrix = pickle.load(open(WEIGHTS, 'rb'))

In [11]:
net = ResNet(weights_matrix, embed_dim = EMBED_DIM)

In [12]:
net.to('cuda')

ResNet(
  (embedding): Embedding(9121, 50, padding_idx=1)
  (init_conv): Conv1d(1, 32, kernel_size=(3, 50), stride=(1,), padding=(1, 0), bias=False)
  (features): Sequential(
    (init_conv): Conv1d(1, 32, kernel_size=(3, 50), stride=(1,), padding=(1, 0), bias=False)
    (init_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (init_relu): LeakyReLU(negative_slope=0.01, inplace=True)
    (resblock1): Sequential(
      (0): SequentialEx(
        (layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
            (1): LeakyReLU(negative_slope=0.01, inplace=True)
            (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): SelfAttention(
              (query): Conv1d(32, 4, kernel_size=(1,), stride=(1,), bias=False)
              (key): Conv1d(32, 4, kernel_size=(1,), stride=(1,), bias=False)
              (value): C

In [13]:
learn = Learner(data=data, model=net, wd=0.1, loss_func=CrossEntropyFlat(),  metrics=[accuracy, FBeta(average='macro',beta=1)])

# Result - Train

In [14]:
learn.fit_one_cycle(15, 1e-03, moms=(0.85, 0.7))

epoch,train_loss,valid_loss,accuracy,f_beta,time
0,2.365358,0.694803,0.476226,0.468355,01:25
1,1.228214,0.689014,0.560090,0.362507,01:25
2,0.757421,0.695238,0.438787,0.304970,01:26
3,0.694149,0.690137,0.561026,0.359396,01:25
4,0.662767,0.815699,0.438974,0.305061,01:26
5,0.493297,0.605271,0.673718,0.661547,01:25
6,0.374684,0.451997,0.789030,0.788682,01:25
7,0.300146,0.389962,0.845189,0.844028,01:25
8,0.245892,0.382954,0.830588,0.820320,01:26
9,0.176367,0.392744,0.821041,0.821001,01:25


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
torch.save(net.state_dict(), './Model/Headline')

# Testing

In [16]:
PADDING = 128 
DATASET = 'RedditMain.csv'  
COL = 'comment'   #headline for Headlines, comment for Reddit Main/Pol

In [17]:
data = (TextList.from_csv(DATASET_PATH, DATASET, cols=COL, processor=processor))

In [18]:
data = data.split_from_df(col='valid').label_from_df(cols=0).databunch()

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [19]:
net.load_state_dict(torch.load('./Model/Headline'))

<All keys matched successfully>

In [20]:
net.to('cuda')

ResNet(
  (embedding): Embedding(9121, 50, padding_idx=1)
  (init_conv): Conv1d(1, 32, kernel_size=(3, 50), stride=(1,), padding=(1, 0), bias=False)
  (features): Sequential(
    (init_conv): Conv1d(1, 32, kernel_size=(3, 50), stride=(1,), padding=(1, 0), bias=False)
    (init_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (init_relu): LeakyReLU(negative_slope=0.01, inplace=True)
    (resblock1): Sequential(
      (0): SequentialEx(
        (layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
            (1): LeakyReLU(negative_slope=0.01, inplace=True)
            (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): SelfAttention(
              (query): Conv1d(32, 4, kernel_size=(1,), stride=(1,), bias=False)
              (key): Conv1d(32, 4, kernel_size=(1,), stride=(1,), bias=False)
              (value): C

# Result - Test

In [21]:
net.eval()
learn = Learner(data=data, model=net, loss_func=CrossEntropyFlat(),  metrics=[accuracy, FBeta(average='macro',beta=1)])
learn.fit_one_cycle(3, 1e-03, moms=(0.85, 0.75))

epoch,train_loss,valid_loss,accuracy,f_beta,time
0,0.617002,0.636939,0.653623,0.651248,14:04


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


KeyboardInterrupt: ignored

In [22]:
torch.save(net.state_dict(), './Model/Final')